# Magic Methods (Dunder Methods) in Python

---

## Table of Contents
1. What are Magic Methods?
2. Object Representation: __str__, __repr__
3. Object Creation: __new__, __init__, __del__
4. Comparison Methods
5. Arithmetic Operators
6. Container Methods
7. Callable Objects: __call__
8. Context Managers: __enter__, __exit__
9. Attribute Access
10. Other Useful Magic Methods
11. Key Points
12. Practice Exercises

---

## 1. What are Magic Methods?

**Magic Methods** (also called **dunder methods** for "double underscore") are special methods that Python calls automatically in certain situations.

**Key Points:**
- Named with double underscores: `__method__`
- Called implicitly by Python (not directly by user)
- Allow customization of built-in behavior
- Enable operator overloading

In [ ]:
# Magic methods are everywhere!
x = 5

# These are equivalent:
print(x + 3)           # Uses __add__
print(x.__add__(3))    # Direct call

print(len("hello"))           # Uses __len__
print("hello".__len__())      # Direct call

print(str(42))         # Uses __str__
print((42).__str__())  # Direct call

In [ ]:
# List of common magic methods
class SampleClass:
    pass

# See what magic methods are available
magic = [m for m in dir(SampleClass) if m.startswith('__') and m.endswith('__')]
print("Available magic methods:")
for i, m in enumerate(magic, 1):
    print(f"{i:2}. {m}")

---

## 2. Object Representation: __str__, __repr__

In [ ]:
# Without __str__ and __repr__
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

p = Point(3, 4)
print(p)        # Unhelpful output
print(repr(p))  # Also unhelpful

In [ ]:
# With __str__ and __repr__
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __str__(self):
        """User-friendly string (used by print, str())"""
        return f"Point at ({self.x}, {self.y})"
    
    def __repr__(self):
        """Developer string (unambiguous, ideally valid Python)"""
        return f"Point({self.x}, {self.y})"

p = Point(3, 4)
print(str(p))   # Calls __str__
print(repr(p))  # Calls __repr__
print(p)        # Calls __str__

# In collections, __repr__ is used
points = [Point(1, 2), Point(3, 4)]
print(points)  # Uses __repr__ for each item

In [ ]:
# If only __repr__ is defined, it's used for both
class Vector:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __repr__(self):
        return f"Vector({self.x}, {self.y})"

v = Vector(5, 6)
print(str(v))   # Falls back to __repr__
print(repr(v))

---

## 3. Object Creation: __new__, __init__, __del__

In [ ]:
# __new__ vs __init__
class MyClass:
    def __new__(cls, *args, **kwargs):
        """Creates the instance (rarely overridden)"""
        print(f"__new__ called with args: {args}")
        instance = super().__new__(cls)
        return instance
    
    def __init__(self, value):
        """Initializes the instance (commonly used)"""
        print(f"__init__ called with value: {value}")
        self.value = value

obj = MyClass(42)

In [ ]:
# Singleton pattern using __new__
class Singleton:
    _instance = None
    
    def __new__(cls):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        return cls._instance

s1 = Singleton()
s2 = Singleton()
print(f"Same instance: {s1 is s2}")

In [ ]:
# __del__ (destructor)
class Resource:
    def __init__(self, name):
        self.name = name
        print(f"Resource '{name}' created")
    
    def __del__(self):
        """Called when object is garbage collected"""
        print(f"Resource '{self.name}' destroyed")

r = Resource("file.txt")
del r  # Explicitly delete
print("After del")

---

## 4. Comparison Methods

In [ ]:
# Comparison magic methods
class Money:
    def __init__(self, amount, currency="USD"):
        self.amount = amount
        self.currency = currency
    
    def __repr__(self):
        return f"Money({self.amount}, '{self.currency}')"
    
    def __eq__(self, other):
        """==  Equality"""
        if isinstance(other, Money):
            return self.amount == other.amount and self.currency == other.currency
        return NotImplemented
    
    def __ne__(self, other):
        """!=  Not equal"""
        result = self.__eq__(other)
        if result is NotImplemented:
            return result
        return not result
    
    def __lt__(self, other):
        """<   Less than"""
        if isinstance(other, Money) and self.currency == other.currency:
            return self.amount < other.amount
        return NotImplemented
    
    def __le__(self, other):
        """<=  Less than or equal"""
        if isinstance(other, Money) and self.currency == other.currency:
            return self.amount <= other.amount
        return NotImplemented
    
    def __gt__(self, other):
        """>   Greater than"""
        if isinstance(other, Money) and self.currency == other.currency:
            return self.amount > other.amount
        return NotImplemented
    
    def __ge__(self, other):
        """>=  Greater than or equal"""
        if isinstance(other, Money) and self.currency == other.currency:
            return self.amount >= other.amount
        return NotImplemented

m1 = Money(100)
m2 = Money(100)
m3 = Money(200)

print(f"m1 == m2: {m1 == m2}")
print(f"m1 < m3: {m1 < m3}")
print(f"m3 > m1: {m3 > m1}")
print(f"m1 <= m2: {m1 <= m2}")

In [ ]:
# Using functools.total_ordering to reduce boilerplate
from functools import total_ordering

@total_ordering
class Student:
    def __init__(self, name, grade):
        self.name = name
        self.grade = grade
    
    def __repr__(self):
        return f"Student('{self.name}', {self.grade})"
    
    # Only need __eq__ and one of __lt__, __le__, __gt__, __ge__
    def __eq__(self, other):
        return self.grade == other.grade
    
    def __lt__(self, other):
        return self.grade < other.grade

s1 = Student("Alice", 85)
s2 = Student("Bob", 90)
s3 = Student("Charlie", 85)

print(f"s1 < s2: {s1 < s2}")
print(f"s2 > s1: {s2 > s1}")  # Generated automatically
print(f"s1 == s3: {s1 == s3}")
print(f"s1 >= s3: {s1 >= s3}")  # Generated automatically

# Now we can sort!
students = [s2, s1, s3]
print(f"Sorted: {sorted(students)}")

In [ ]:
# __hash__ for use in sets and dict keys
class HashablePoint:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y
    
    def __hash__(self):
        return hash((self.x, self.y))
    
    def __repr__(self):
        return f"Point({self.x}, {self.y})"

# Can use in sets and as dict keys
points = {HashablePoint(1, 2), HashablePoint(3, 4), HashablePoint(1, 2)}
print(f"Set of points: {points}")  # Duplicates removed

point_dict = {HashablePoint(0, 0): "origin", HashablePoint(1, 0): "unit x"}
print(f"Dict: {point_dict}")

---

## 5. Arithmetic Operators

In [ ]:
# Arithmetic magic methods
class Vector:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __repr__(self):
        return f"Vector({self.x}, {self.y})"
    
    def __add__(self, other):
        """ + Addition"""
        if isinstance(other, Vector):
            return Vector(self.x + other.x, self.y + other.y)
        return NotImplemented
    
    def __sub__(self, other):
        """ - Subtraction"""
        if isinstance(other, Vector):
            return Vector(self.x - other.x, self.y - other.y)
        return NotImplemented
    
    def __mul__(self, scalar):
        """ * Multiplication (vector * scalar)"""
        if isinstance(scalar, (int, float)):
            return Vector(self.x * scalar, self.y * scalar)
        return NotImplemented
    
    def __rmul__(self, scalar):
        """ * Multiplication (scalar * vector)"""
        return self.__mul__(scalar)
    
    def __truediv__(self, scalar):
        """ / Division"""
        if isinstance(scalar, (int, float)):
            return Vector(self.x / scalar, self.y / scalar)
        return NotImplemented
    
    def __neg__(self):
        """ - Negation (unary)"""
        return Vector(-self.x, -self.y)
    
    def __pos__(self):
        """ + Positive (unary)"""
        return Vector(self.x, self.y)
    
    def __abs__(self):
        """abs() - Absolute value/magnitude"""
        return (self.x ** 2 + self.y ** 2) ** 0.5

v1 = Vector(3, 4)
v2 = Vector(1, 2)

print(f"v1 + v2 = {v1 + v2}")
print(f"v1 - v2 = {v1 - v2}")
print(f"v1 * 2 = {v1 * 2}")
print(f"2 * v1 = {2 * v1}")
print(f"v1 / 2 = {v1 / 2}")
print(f"-v1 = {-v1}")
print(f"abs(v1) = {abs(v1)}")

In [ ]:
# In-place operators (augmented assignment)
class Counter:
    def __init__(self, value=0):
        self.value = value
    
    def __repr__(self):
        return f"Counter({self.value})"
    
    def __iadd__(self, other):
        """ += In-place addition"""
        self.value += other
        return self
    
    def __isub__(self, other):
        """ -= In-place subtraction"""
        self.value -= other
        return self
    
    def __imul__(self, other):
        """ *= In-place multiplication"""
        self.value *= other
        return self

c = Counter(10)
print(f"Initial: {c}")

c += 5
print(f"After +=5: {c}")

c -= 3
print(f"After -=3: {c}")

c *= 2
print(f"After *=2: {c}")

In [ ]:
# Floor division, modulo, and power
class Number:
    def __init__(self, value):
        self.value = value
    
    def __repr__(self):
        return f"Number({self.value})"
    
    def __floordiv__(self, other):
        """ // Floor division"""
        return Number(self.value // other)
    
    def __mod__(self, other):
        """ % Modulo"""
        return Number(self.value % other)
    
    def __pow__(self, other):
        """ ** Power"""
        return Number(self.value ** other)
    
    def __divmod__(self, other):
        """divmod() - returns (quotient, remainder)"""
        return (self.value // other, self.value % other)

n = Number(17)
print(f"17 // 5 = {n // 5}")
print(f"17 % 5 = {n % 5}")
print(f"17 ** 2 = {n ** 2}")
print(f"divmod(17, 5) = {divmod(n, 5)}")

---

## 6. Container Methods

In [ ]:
# Container magic methods
class CustomList:
    def __init__(self, items=None):
        self._items = list(items) if items else []
    
    def __repr__(self):
        return f"CustomList({self._items})"
    
    def __len__(self):
        """len() - Return length"""
        return len(self._items)
    
    def __getitem__(self, index):
        """obj[index] - Get item"""
        return self._items[index]
    
    def __setitem__(self, index, value):
        """obj[index] = value - Set item"""
        self._items[index] = value
    
    def __delitem__(self, index):
        """del obj[index] - Delete item"""
        del self._items[index]
    
    def __contains__(self, item):
        """item in obj - Membership test"""
        return item in self._items
    
    def __iter__(self):
        """for item in obj - Iteration"""
        return iter(self._items)
    
    def __reversed__(self):
        """reversed(obj) - Reverse iteration"""
        return reversed(self._items)

cl = CustomList([1, 2, 3, 4, 5])

print(f"Length: {len(cl)}")
print(f"Index 2: {cl[2]}")
print(f"Slice [1:4]: {cl[1:4]}")

cl[0] = 10
print(f"After cl[0]=10: {cl}")

print(f"3 in cl: {3 in cl}")

print("Iterating:", end=" ")
for item in cl:
    print(item, end=" ")
print()

print(f"Reversed: {list(reversed(cl))}")

In [ ]:
# Dictionary-like container
class CaseInsensitiveDict:
    """Dictionary with case-insensitive keys."""
    
    def __init__(self):
        self._data = {}
    
    def _normalize_key(self, key):
        return key.lower() if isinstance(key, str) else key
    
    def __repr__(self):
        return f"CaseInsensitiveDict({self._data})"
    
    def __len__(self):
        return len(self._data)
    
    def __getitem__(self, key):
        return self._data[self._normalize_key(key)]
    
    def __setitem__(self, key, value):
        self._data[self._normalize_key(key)] = value
    
    def __delitem__(self, key):
        del self._data[self._normalize_key(key)]
    
    def __contains__(self, key):
        return self._normalize_key(key) in self._data
    
    def __iter__(self):
        return iter(self._data)

cid = CaseInsensitiveDict()
cid["Name"] = "Alice"
cid["AGE"] = 30

print(f"cid['name'] = {cid['name']}")
print(f"cid['NAME'] = {cid['NAME']}")
print(f"'NAME' in cid: {'NAME' in cid}")
print(f"cid: {cid}")

In [ ]:
# __bool__ - truth value
class Queue:
    def __init__(self):
        self._items = []
    
    def __bool__(self):
        """bool(obj) - Returns True if not empty"""
        return len(self._items) > 0
    
    def __len__(self):
        return len(self._items)
    
    def enqueue(self, item):
        self._items.append(item)
    
    def dequeue(self):
        return self._items.pop(0) if self._items else None

q = Queue()

print(f"Empty queue is truthy: {bool(q)}")

q.enqueue("task1")
print(f"Queue with item is truthy: {bool(q)}")

# Use in conditions
while q:
    print(f"Processing: {q.dequeue()}")

---

## 7. Callable Objects: __call__

In [ ]:
# __call__ makes objects callable like functions
class Multiplier:
    def __init__(self, factor):
        self.factor = factor
    
    def __call__(self, value):
        """obj() - Make object callable"""
        return value * self.factor

double = Multiplier(2)
triple = Multiplier(3)

print(f"double(5) = {double(5)}")
print(f"triple(5) = {triple(5)}")

# Check if callable
print(f"Is double callable: {callable(double)}")

In [ ]:
# Stateful callable - counter
class CallCounter:
    def __init__(self, func):
        self.func = func
        self.count = 0
    
    def __call__(self, *args, **kwargs):
        self.count += 1
        print(f"Call #{self.count}")
        return self.func(*args, **kwargs)

@CallCounter
def greet(name):
    return f"Hello, {name}!"

print(greet("Alice"))
print(greet("Bob"))
print(greet("Charlie"))
print(f"Total calls: {greet.count}")

In [ ]:
# Callable class for memoization
class Memoize:
    def __init__(self, func):
        self.func = func
        self.cache = {}
    
    def __call__(self, *args):
        if args in self.cache:
            print(f"Cache hit for {args}")
            return self.cache[args]
        print(f"Computing for {args}")
        result = self.func(*args)
        self.cache[args] = result
        return result

@Memoize
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)

print(f"fib(10) = {fibonacci(10)}")
print(f"\nfib(10) again = {fibonacci(10)}")

---

## 8. Context Managers: __enter__, __exit__

In [ ]:
# Context manager for resource management
class FileManager:
    def __init__(self, filename, mode):
        self.filename = filename
        self.mode = mode
        self.file = None
    
    def __enter__(self):
        """Called when entering 'with' block"""
        print(f"Opening {self.filename}")
        self.file = open(self.filename, self.mode)
        return self.file
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        """Called when exiting 'with' block"""
        print(f"Closing {self.filename}")
        if self.file:
            self.file.close()
        # Return False to propagate exceptions
        return False

# Usage
with FileManager("test.txt", "w") as f:
    f.write("Hello, World!")

print("File operations complete")

In [ ]:
# Timer context manager
import time

class Timer:
    def __init__(self, name="Operation"):
        self.name = name
    
    def __enter__(self):
        self.start = time.time()
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time.time()
        self.elapsed = self.end - self.start
        print(f"{self.name} took {self.elapsed:.4f} seconds")
        return False

with Timer("Sum calculation"):
    total = sum(range(1000000))

with Timer("List comprehension"):
    squares = [x**2 for x in range(10000)]

In [ ]:
# Exception handling in context managers
class SuppressErrors:
    def __init__(self, *exceptions):
        self.exceptions = exceptions
    
    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        if exc_type is not None:
            if issubclass(exc_type, self.exceptions):
                print(f"Suppressed: {exc_type.__name__}: {exc_val}")
                return True  # Suppress the exception
        return False  # Propagate other exceptions

# Suppress ZeroDivisionError
with SuppressErrors(ZeroDivisionError):
    result = 1 / 0
    print("This won't print")

print("Continued after suppressed error")

# Suppress multiple exception types
with SuppressErrors(ZeroDivisionError, ValueError):
    int("not a number")

print("Also continued")

---

## 9. Attribute Access

In [ ]:
# __getattr__, __setattr__, __delattr__
class DynamicAttributes:
    def __init__(self):
        self._data = {}
    
    def __getattr__(self, name):
        """Called when attribute is not found normally"""
        if name.startswith('_'):
            raise AttributeError(f"'{type(self).__name__}' object has no attribute '{name}'")
        return self._data.get(name, f"<undefined: {name}>")
    
    def __setattr__(self, name, value):
        """Called for all attribute assignments"""
        if name.startswith('_'):
            # Allow private attributes to be set normally
            super().__setattr__(name, value)
        else:
            self._data[name] = value
    
    def __delattr__(self, name):
        """Called when deleting an attribute"""
        if name in self._data:
            del self._data[name]
        else:
            raise AttributeError(f"'{name}' not found")

obj = DynamicAttributes()
obj.name = "Alice"
obj.age = 30

print(f"name: {obj.name}")
print(f"age: {obj.age}")
print(f"undefined: {obj.undefined_attr}")

del obj.name
print(f"After delete: {obj.name}")

In [ ]:
# __getattribute__ - called for EVERY attribute access
class LoggedAccess:
    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            object.__setattr__(self, key, value)
    
    def __getattribute__(self, name):
        """Called for every attribute access"""
        print(f"Accessing: {name}")
        return object.__getattribute__(self, name)

obj = LoggedAccess(x=10, y=20)
print(obj.x)
print(obj.y)

In [ ]:
# __dir__ - customize dir() output
class Plugin:
    def __init__(self):
        self._internal = "private"
        self.name = "MyPlugin"
        self.version = "1.0"
    
    def run(self):
        return "Running"
    
    def __dir__(self):
        """Customize what dir() returns"""
        # Only show public attributes
        return [key for key in self.__dict__ if not key.startswith('_')]

plugin = Plugin()
print(f"dir(plugin): {dir(plugin)}")

---

## 10. Other Useful Magic Methods

In [ ]:
# __format__ - customize format()
class Temperature:
    def __init__(self, celsius):
        self.celsius = celsius
    
    def __format__(self, spec):
        if spec == 'f':  # Fahrenheit
            return f"{self.celsius * 9/5 + 32:.1f}F"
        elif spec == 'k':  # Kelvin
            return f"{self.celsius + 273.15:.1f}K"
        else:  # Celsius (default)
            return f"{self.celsius:.1f}C"

temp = Temperature(25)
print(f"Celsius: {temp}")
print(f"Fahrenheit: {temp:f}")
print(f"Kelvin: {temp:k}")

In [ ]:
# __copy__ and __deepcopy__
import copy

class Config:
    def __init__(self, settings):
        self.settings = settings
        self._cache = {}  # Don't copy cache
    
    def __copy__(self):
        """Shallow copy"""
        print("Shallow copying Config")
        new = Config.__new__(Config)
        new.settings = self.settings  # Same reference
        new._cache = {}  # Fresh cache
        return new
    
    def __deepcopy__(self, memo):
        """Deep copy"""
        print("Deep copying Config")
        new = Config.__new__(Config)
        new.settings = copy.deepcopy(self.settings, memo)
        new._cache = {}  # Fresh cache
        return new

original = Config({"debug": True, "db": {"host": "localhost"}})
original._cache["key"] = "cached"

shallow = copy.copy(original)
print(f"Shallow - same settings dict: {original.settings is shallow.settings}")

deep = copy.deepcopy(original)
print(f"Deep - same settings dict: {original.settings is deep.settings}")

In [ ]:
# __slots__ - memory optimization
class RegularClass:
    def __init__(self, x, y):
        self.x = x
        self.y = y

class SlottedClass:
    __slots__ = ['x', 'y']  # No __dict__
    
    def __init__(self, x, y):
        self.x = x
        self.y = y

regular = RegularClass(1, 2)
slotted = SlottedClass(1, 2)

print(f"Regular has __dict__: {hasattr(regular, '__dict__')}")
print(f"Slotted has __dict__: {hasattr(slotted, '__dict__')}")

# Cannot add new attributes to slotted class
regular.z = 3  # Works
try:
    slotted.z = 3  # Fails
except AttributeError as e:
    print(f"Cannot add attribute: {e}")

In [ ]:
# __sizeof__ - customize memory size
import sys

class DataContainer:
    def __init__(self, data):
        self.data = data
    
    def __sizeof__(self):
        """Return memory size in bytes"""
        base_size = object.__sizeof__(self)
        data_size = sys.getsizeof(self.data)
        return base_size + data_size

container = DataContainer([1, 2, 3, 4, 5])
print(f"Size of container: {sys.getsizeof(container)} bytes")

---

## 11. Key Points

**Representation:**
- `__str__`: User-friendly string (print, str())
- `__repr__`: Developer string (repr(), debugging)

**Object Lifecycle:**
- `__new__`: Creates instance (rarely overridden)
- `__init__`: Initializes instance (commonly used)
- `__del__`: Destructor (avoid relying on it)

**Comparison:**
- `__eq__`, `__ne__`, `__lt__`, `__le__`, `__gt__`, `__ge__`
- `__hash__`: For use in sets/dicts
- Use `@total_ordering` to reduce boilerplate

**Arithmetic:**
- `__add__`, `__sub__`, `__mul__`, `__truediv__`, etc.
- `__radd__` (right-hand), `__iadd__` (in-place)
- `__neg__`, `__pos__`, `__abs__` (unary)

**Containers:**
- `__len__`, `__getitem__`, `__setitem__`, `__delitem__`
- `__contains__`, `__iter__`, `__reversed__`
- `__bool__`: Truth value testing

**Callable:**
- `__call__`: Make objects callable like functions

**Context Managers:**
- `__enter__`, `__exit__`: For 'with' statement

**Attribute Access:**
- `__getattr__`: Called when attribute not found
- `__setattr__`, `__delattr__`: Set/delete attributes
- `__getattribute__`: Called for every attribute access

---

## 12. Practice Exercises

In [ ]:
# Exercise 1: Create a Fraction class with:
# - __init__(numerator, denominator)
# - __str__, __repr__
# - __add__, __sub__, __mul__, __truediv__
# - __eq__, __lt__ (with @total_ordering)
# - Simplify fractions using GCD

from functools import total_ordering

@total_ordering
class Fraction:
    pass

# Test:
# f1 = Fraction(1, 2)
# f2 = Fraction(1, 4)
# print(f1 + f2)  # 3/4
# print(f1 * f2)  # 1/8

In [ ]:
# Exercise 2: Create a Matrix class with:
# - __init__ taking 2D list
# - __repr__
# - __getitem__ for matrix[row][col] or matrix[row, col]
# - __add__ for matrix addition
# - __mul__ for matrix-scalar multiplication

class Matrix:
    pass

# Test:
# m = Matrix([[1, 2], [3, 4]])
# print(m[0, 1])  # 2
# print(m * 2)    # [[2, 4], [6, 8]]

In [ ]:
# Exercise 3: Create a Stack class with:
# - __len__
# - __bool__ (True if not empty)
# - __iter__ (iterate from top to bottom)
# - __contains__
# - push(), pop(), peek() methods

class Stack:
    pass

# Test:
# s = Stack()
# s.push(1); s.push(2); s.push(3)
# print(2 in s)  # True
# for item in s: print(item)  # 3, 2, 1

In [ ]:
# Exercise 4: Create a RateLimiter context manager
# - Limits operations per second
# - Tracks calls and elapsed time
# - __enter__ starts timing
# - __exit__ prints stats

class RateLimiter:
    pass

# Test:
# with RateLimiter(max_calls=5) as rl:
#     for i in range(10):
#         rl.call()  # Only 5 should succeed

In [ ]:
# Exercise 5: Create a Cached callable class
# - Decorator that caches function results
# - __call__ checks cache before computing
# - cache_info() method shows hits/misses
# - clear_cache() method

class Cached:
    pass

# Test:
# @Cached
# def expensive(n):
#     return sum(range(n))
# expensive(1000); expensive(1000)  # Second call from cache
# print(expensive.cache_info())  # hits=1, misses=1

---

## Solutions

In [ ]:
# Solution 1:
from functools import total_ordering
from math import gcd

@total_ordering
class Fraction:
    def __init__(self, numerator, denominator):
        if denominator == 0:
            raise ValueError("Denominator cannot be zero")
        common = gcd(numerator, denominator)
        self.num = numerator // common
        self.den = denominator // common
        if self.den < 0:
            self.num = -self.num
            self.den = -self.den
    
    def __str__(self):
        return f"{self.num}/{self.den}"
    
    def __repr__(self):
        return f"Fraction({self.num}, {self.den})"
    
    def __add__(self, other):
        new_num = self.num * other.den + other.num * self.den
        new_den = self.den * other.den
        return Fraction(new_num, new_den)
    
    def __sub__(self, other):
        new_num = self.num * other.den - other.num * self.den
        new_den = self.den * other.den
        return Fraction(new_num, new_den)
    
    def __mul__(self, other):
        return Fraction(self.num * other.num, self.den * other.den)
    
    def __truediv__(self, other):
        return Fraction(self.num * other.den, self.den * other.num)
    
    def __eq__(self, other):
        return self.num == other.num and self.den == other.den
    
    def __lt__(self, other):
        return self.num * other.den < other.num * self.den

f1 = Fraction(1, 2)
f2 = Fraction(1, 4)
print(f"f1 + f2 = {f1 + f2}")
print(f"f1 - f2 = {f1 - f2}")
print(f"f1 * f2 = {f1 * f2}")
print(f"f1 / f2 = {f1 / f2}")
print(f"f1 > f2: {f1 > f2}")

In [ ]:
# Solution 2:
class Matrix:
    def __init__(self, data):
        self.data = data
        self.rows = len(data)
        self.cols = len(data[0]) if data else 0
    
    def __repr__(self):
        rows = ['[' + ', '.join(map(str, row)) + ']' for row in self.data]
        return 'Matrix([' + ',\n        '.join(rows) + '])'
    
    def __getitem__(self, key):
        if isinstance(key, tuple):
            row, col = key
            return self.data[row][col]
        return self.data[key]
    
    def __add__(self, other):
        if self.rows != other.rows or self.cols != other.cols:
            raise ValueError("Matrix dimensions must match")
        result = [
            [self.data[i][j] + other.data[i][j] for j in range(self.cols)]
            for i in range(self.rows)
        ]
        return Matrix(result)
    
    def __mul__(self, scalar):
        result = [
            [self.data[i][j] * scalar for j in range(self.cols)]
            for i in range(self.rows)
        ]
        return Matrix(result)
    
    def __rmul__(self, scalar):
        return self.__mul__(scalar)

m1 = Matrix([[1, 2], [3, 4]])
m2 = Matrix([[5, 6], [7, 8]])

print(f"m1[0, 1] = {m1[0, 1]}")
print(f"m1 + m2 = {m1 + m2}")
print(f"m1 * 2 = {m1 * 2}")

In [ ]:
# Solution 3:
class Stack:
    def __init__(self):
        self._items = []
    
    def __len__(self):
        return len(self._items)
    
    def __bool__(self):
        return len(self._items) > 0
    
    def __iter__(self):
        return reversed(self._items)
    
    def __contains__(self, item):
        return item in self._items
    
    def __repr__(self):
        return f"Stack({self._items})"
    
    def push(self, item):
        self._items.append(item)
    
    def pop(self):
        if not self._items:
            raise IndexError("Pop from empty stack")
        return self._items.pop()
    
    def peek(self):
        if not self._items:
            raise IndexError("Peek from empty stack")
        return self._items[-1]

s = Stack()
s.push(1)
s.push(2)
s.push(3)

print(f"Stack: {s}")
print(f"Length: {len(s)}")
print(f"2 in s: {2 in s}")
print(f"Peek: {s.peek()}")
print("Iterating:", list(s))

In [ ]:
# Solution 4:
import time

class RateLimiter:
    def __init__(self, max_calls):
        self.max_calls = max_calls
        self.calls = 0
        self.successful = 0
        self.denied = 0
    
    def __enter__(self):
        self.start_time = time.time()
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        elapsed = time.time() - self.start_time
        print(f"Stats: {self.successful} successful, {self.denied} denied")
        print(f"Time elapsed: {elapsed:.4f} seconds")
        return False
    
    def call(self):
        self.calls += 1
        if self.successful < self.max_calls:
            self.successful += 1
            print(f"Call {self.calls}: Allowed")
            return True
        else:
            self.denied += 1
            print(f"Call {self.calls}: Denied (limit reached)")
            return False

with RateLimiter(max_calls=3) as rl:
    for i in range(5):
        rl.call()

In [ ]:
# Solution 5:
class Cached:
    def __init__(self, func):
        self.func = func
        self.cache = {}
        self.hits = 0
        self.misses = 0
    
    def __call__(self, *args, **kwargs):
        key = (args, tuple(sorted(kwargs.items())))
        if key in self.cache:
            self.hits += 1
            return self.cache[key]
        self.misses += 1
        result = self.func(*args, **kwargs)
        self.cache[key] = result
        return result
    
    def cache_info(self):
        return f"CacheInfo(hits={self.hits}, misses={self.misses}, size={len(self.cache)})"
    
    def clear_cache(self):
        self.cache.clear()
        self.hits = 0
        self.misses = 0

@Cached
def expensive(n):
    return sum(range(n))

print(f"expensive(1000) = {expensive(1000)}")
print(f"expensive(1000) = {expensive(1000)}")
print(f"expensive(2000) = {expensive(2000)}")
print(expensive.cache_info())